In [7]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from keras.layers import Input, Dense
from keras.losses import MeanSquaredError
from keras import Model
import pandas as pd

In [3]:
train = pd.read_csv("train.csv").dropna()
test = pd.read_csv("test.csv").dropna()

def append_sentiment(df):
    return df.sentiment.str.strip() + " " + df.text.str.strip()

train["text"] = append_sentiment(train)
train["selected_from"] = train.apply(lambda x: x.text.find(x.selected_text), axis=1)
train["selected_to"] = train.apply(lambda x: x.selected_from + len(x.selected_text), axis=1)
x_train = train.text
y_train = train[["selected_from", "selected_to"]]
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.15)

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
embedding = tokenizer(x_train.to_list(), padding="max_length", max_length=256, truncation=True)

In [9]:
inputs = Input(shape=(256,))
outputs = Dense(1, activation="relu")(inputs)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss=MeanSquaredError, metrics=["accuracy"])
model.summary()

2025-01-15 16:49:11.453494: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257 (1.00 KB)

 Trainable params: 257 (1.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(embedding["input_ids"], y_train, batch_size=64, epochs=2)